# Master Discogs Database

In [12]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from searchUtils import findExt, findSubExt, findPatternExt
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from artist import artist
from albums import albums
from album import album, albumURLInfo
from time import sleep

from masterdb import discConv, isKnownAlbum, directoryName

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-22 20:34:10.093439


In [32]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
art  = artist()
albs = albums(disc)
alb  = album()
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Create Dictionary Lookup Files

## Artist ID --> Ref and Name

In [33]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistIDMap
createArtistIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Jan 23, 2020 21:07:43 for Creating Artist DBs
/Volumes/Music/Discog/artists-db/metadata/0-Metadata.p  	0 7041
/Volumes/Music/Discog/artists-db/metadata/1-Metadata.p  	1 14328
/Volumes/Music/Discog/artists-db/metadata/10-Metadata.p  	2 21402
/Volumes/Music/Discog/artists-db/metadata/11-Metadata.p  	3 28578
/Volumes/Music/Discog/artists-db/metadata/12-Metadata.p  	4 35761
/Volumes/Music/Discog/artists-db/metadata/13-Metadata.p  	5 42987
/Volumes/Music/Discog/artists-db/metadata/14-Metadata.p  	6 49987
/Volumes/Music/Discog/artists-db/metadata/15-Metadata.p  	7 57111
/Volumes/Music/Discog/artists-db/metadata/16-Metadata.p  	8 64178
/Volumes/Music/Discog/artists-db/metadata/17-Metadata.p  	9 71344
/Volumes/Music/Discog/artists-db/metadata/18-Metadata.p  	10 78509
/Volumes/Music/Discog/artists-db/metadata/19-Metadata.p  	11 85638
/Volumes/Music/Discog/artists-db/metadata/2-Metadata.p 

## Artist ID --> Albums

In [34]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Jan 23, 2020 21:09:24 for Creating Artist DBs
/Volumes/Music/Discog/artists-db/metadata/0-MediaMetadata.p 	7041      3615      19361     
/Volumes/Music/Discog/artists-db/metadata/1-MediaMetadata.p 	14328     7310      39819     
/Volumes/Music/Discog/artists-db/metadata/10-MediaMetadata.p 	21402     10954     59663     
/Volumes/Music/Discog/artists-db/metadata/11-MediaMetadata.p 	28578     14627     79593     
/Volumes/Music/Discog/artists-db/metadata/12-MediaMetadata.p 	35761     18338     99557     
/Volumes/Music/Discog/artists-db/metadata/13-MediaMetadata.p 	42987     22008     119656    
/Volumes/Music/Discog/artists-db/metadata/14-MediaMetadata.p 	49987     25620     139110    
/Volumes/Music/Discog/artists-db/metadata/15-MediaMetadata.p 	57111     29252     159539    
/Volumes/Music/Discog/artists-db/metadata/16-MediaMetadata.p 	64178     32860     179675    
/Volumes/Mu

/Volumes/Music/Discog/artists-db/metadata/87-MediaMetadata.p 	619627    316384    1743570   
/Volumes/Music/Discog/artists-db/metadata/88-MediaMetadata.p 	626689    319941    1763642   
/Volumes/Music/Discog/artists-db/metadata/89-MediaMetadata.p 	633957    323636    1783925   
/Volumes/Music/Discog/artists-db/metadata/9-MediaMetadata.p 	641108    327267    1804269   
/Volumes/Music/Discog/artists-db/metadata/90-MediaMetadata.p 	648104    330807    1824193   
/Volumes/Music/Discog/artists-db/metadata/91-MediaMetadata.p 	655209    334450    1843984   
/Volumes/Music/Discog/artists-db/metadata/92-MediaMetadata.p 	662470    338146    1864919   
/Volumes/Music/Discog/artists-db/metadata/93-MediaMetadata.p 	669595    341806    1885264   
/Volumes/Music/Discog/artists-db/metadata/94-MediaMetadata.p 	676710    345385    1904872   
/Volumes/Music/Discog/artists-db/metadata/95-MediaMetadata.p 	683911    349054    1924953   
/Volumes/Music/Discog/artists-db/metadata/96-MediaMetadata.p 	690988   

## Artist ID --> Genre, Style, Artists Lookup Table

In [35]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Jan 23, 2020 21:27:15 for Creating Artist DBs
/Volumes/Music/Discog/albums-db/metadata/0-ArtistMetadata.p 	4771
/Volumes/Music/Discog/albums-db/metadata/1-ArtistMetadata.p 	9481
/Volumes/Music/Discog/albums-db/metadata/10-ArtistMetadata.p 	16165
/Volumes/Music/Discog/albums-db/metadata/11-ArtistMetadata.p 	22936
/Volumes/Music/Discog/albums-db/metadata/12-ArtistMetadata.p 	29738
/Volumes/Music/Discog/albums-db/metadata/13-ArtistMetadata.p 	33792
/Volumes/Music/Discog/albums-db/metadata/14-ArtistMetadata.p 	40028
/Volumes/Music/Discog/albums-db/metadata/15-ArtistMetadata.p 	46750
/Volumes/Music/Discog/albums-db/metadata/16-ArtistMetadata.p 	52022
/Volumes/Music/Discog/albums-db/metadata/17-ArtistMetadata.p 	56703
/Volumes/Music/Discog/albums-db/metadata/18-ArtistMetadata.p 	63415
/Volumes/Music/Discog/albums-db/metadata/19-ArtistMetadata.p 	69777
/Volumes/Music/Discog/albums-db/me

## Album ID --> Name, Ref, Artists Lookup Table

In [36]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createAlbumIDMap
createAlbumIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Jan 23, 2020 21:31:37 for Creating Artist DBs
/Volumes/Music/Discog/albums-db/metadata/0-ArtistAlbums.p 	21825
/Volumes/Music/Discog/albums-db/metadata/1-ArtistAlbums.p 	41453
/Volumes/Music/Discog/albums-db/metadata/10-ArtistAlbums.p 	67791
/Volumes/Music/Discog/albums-db/metadata/11-ArtistAlbums.p 	89756
/Volumes/Music/Discog/albums-db/metadata/12-ArtistAlbums.p 	116582
/Volumes/Music/Discog/albums-db/metadata/13-ArtistAlbums.p 	129077
/Volumes/Music/Discog/albums-db/metadata/14-ArtistAlbums.p 	152458
/Volumes/Music/Discog/albums-db/metadata/15-ArtistAlbums.p 	174344
/Volumes/Music/Discog/albums-db/metadata/16-ArtistAlbums.p 	192435
/Volumes/Music/Discog/albums-db/metadata/17-ArtistAlbums.p 	206069
/Volumes/Music/Discog/albums-db/metadata/18-ArtistAlbums.p 	230543
/Volumes/Music/Discog/albums-db/metadata/19-ArtistAlbums.p 	250792
/Volumes/Music/Discog/albums-db/metadata/2-Artis

# Master Lookup Tests

In [ ]:
%load_ext autoreload
%autoreload
from lookup import testLookupMaps
testLookupMaps(disc)

# Pandas DB

## Slim Artist DB

In [37]:
from masterdb import getSlimArtistDB

In [38]:
discSlimArtistDB = getSlimArtistDB(disc)
discSlimArtistDB.head()

Current Time is Fri Jan 24, 2020 07:19:12 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 16.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Creating Pandas DataFrame for 712336 Artists
	Shape --> (712336, 1)
  Finding Real Artist Name
	Shape --> (712336, 3)
  Removing None Artist
	Shape --> (712336, 3)
  Finding Disc Artist Name
	Shape --> (712336, 4)
DataFrame Shape is (712336, 4)
Current Time is Fri Jan 24, 2020 07:19:58 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 45 seconds.
Saving Master Artist DB File
  --> This file is 57.0MB.


,Name,Artist,Num,DiscArtist
1000,Dave Clarke,Dave Clarke,NaN,Dave Clarke
1000500,Club Pulse,Club Pulse,NaN,Club Pulse
100200,Dike,Dike,NaN,Dike
1002000,Larry Stokes,Larry Stokes,NaN,Larry Stokes
1002600,Gidd Sanchez,Gidd Sanchez,NaN,Gidd Sanchez


## Artist DB

In [19]:
from masterdb import getArtistDB

In [20]:
discArtistDB = getArtistDB(disc)
discArtistDB.head()

Current Time is Wed Jan 22, 2020 20:51:53 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 16.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToVariations.p
  --> This file is 26.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToVariations.p
Creating Pandas DataFrame for 712336 Artists
	Shape --> (712336, 1)
  Joining Ref
	Shape --> (712336, 2)
  Joining Variations
	Shape --> (712336, 3)
  Finding Real Artist Name
	Shape --> (712336, 6)
DataFrame Shape is (712336, 6)
Current Time is Wed Jan 22, 2020 20:52:30 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 36 seconds.
Saving Mas

,Name,Ref,Variations,Known,Artist,Num
1000,Dave Clarke,/artist/1000-Dave-Clarke,"[Dave Clarke, 7 Red 7, Clarck, Clarke, Clarke,...",True,Dave Clarke,NaN
1000500,Club Pulse,/artist/1000500-Club-Pulse,[Club Pulse],True,Club Pulse,NaN
100200,Dike,/artist/100200-Dike,"[Dike, D.I.K.E., Dike D, Uchegdu]",True,Dike,NaN
1002000,Larry Stokes,/artist/1002000-Larry-Stokes,[Larry Stokes],True,Larry Stokes,NaN
1002600,Gidd Sanchez,/artist/1002600-Gidd-Sanchez,[Gidd Sanchez],True,Gidd Sanchez,NaN


## Artist Metadata DB

In [39]:
from masterdb import getArtistMetadataDB

In [40]:
discArtistMetadataDB = getArtistMetadataDB(disc)
discArtistMetadataDB.head()

Current Time is Fri Jan 24, 2020 07:20:08 for 
=================================== Creating Artist Metadata DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToGenre.p
  --> This file is 12.1MB.
Loading /Volumes/Music/Discog/db/ArtistIDToGenre.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToStyle.p
  --> This file is 17.4MB.
Loading /Volumes/Music/Discog/db/ArtistIDToStyle.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToCollaborations.p
  --> This file is 37.7MB.
Loading /Volumes/Music/Discog/db/ArtistIDToCollaborations.p
Creating Pandas DataFrame for 572424 Artists
	Shape --> (572424, 1)
  Joining Style
	Shape --> (572424, 2)
  Joining Collaboration
	Shape --> (572424, 3)
DataFrame Shape is (572424, 3)
Current Time is Fri Jan 24, 2020 07:21:41 for Done with 
=================================== Creating Artist Metadata DB ===================================
Process [{0}] took 1.6 minutes.
Saving Master Artist M

,Genre,Style,Collaboration
1000,"{'Electronic': 55, 'Hip Hop': 1, 'Non-Music': ...","{'Techno': 46, 'Electro': 17, 'Downtempo': 1, ...","{'Dave Clarke': 54, 'Bang The Future': 1, 'Mr...."
1000500,{'Electronic': 1},{'Progressive Trance': 1},{'Club Pulse': 1}
100200,{'Hip Hop': 11},"{'Conscious': 7, 'Pop Rap': 2, '': 3}","{'Dike': 11, 'ABS (2)': 2, 'Creutzfeld&Jakob':..."
1002000,{},{},{}
1005400,{'Rock': 5},"{'Alternative Rock': 1, 'Indie Rock': 1, 'Math...","{'Gapeseed': 5, 'Gerling': 1}"


## Artist Albums DB

In [41]:
from masterdb import getArtistAlbumsDB

In [42]:
discArtistAlbumsDB = getArtistAlbumsDB(disc)
discArtistAlbumsDB.head()

Current Time is Fri Jan 24, 2020 07:22:21 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
  --> This file is 188.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
Creating Pandas DataFrame for 712336 Artists
	Shape --> (712336, 1)
DataFrame Shape is (712336, 1)
Current Time is Fri Jan 24, 2020 07:24:09 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 1.8 minutes.


,Albums
1000,"{'Albums': {'44815': 'Archive One', '44831': '..."
1000500,{'Miscellaneous': {'1178875': 'Peak Controller'}}
100200,"{'Albums': {'50511': 'PottpÃ¼ree'}, 'Singles &..."
1002000,{'Instruments & Performance': {'1002108': 'Are...
1002600,{'Vocals': {'1180905': 'Quiero'}}


## Albums DB

In [26]:
from masterdb import getAlbumDB

In [27]:
discAlbumDB = getAlbumDB(disc)
discAlbumDB.head()

Current Time is Wed Jan 22, 2020 21:10:30 for 
=================================== Creating Artist Album DB ===================================
Loading AlbumID Data
Loading data from /Volumes/Music/Discog/db/AlbumIDToName.p
  --> This file is 39.7MB.
Loading /Volumes/Music/Discog/db/AlbumIDToName.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToRef.p
  --> This file is 55.9MB.
Loading /Volumes/Music/Discog/db/AlbumIDToRef.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToArtists.p
  --> This file is 22.8MB.
Loading /Volumes/Music/Discog/db/AlbumIDToArtists.p
Creating Pandas DataFrame for 1695744 Albums
	Shape --> (1695744, 1)
  Joining Ref
	Shape --> (1695744, 2)
  Joining Artists
	Shape --> (1695744, 3)
DataFrame Shape is (1695744, 3)
Current Time is Wed Jan 22, 2020 21:11:41 for Done with 
=================================== Creating Artist Album DB ===================================
Process [{0}] took 1.2 minutes.


,Name,Ref,Artists
1012077,Live,/Dave-Clarke-Live/master/1012077,[1000]
1258261,Walls Of Genius And Miracle,/Walls-Of-Genius-and-Miracle-Walls-Of-Genius-A...,"[1009227, 182635, 1000]"
2546978,RA.EX007 Dave Clarke,/Dave-Clarke-RAEX007-Dave-Clarke/release/2546978,[1000]
44815,Take Two,/6th-Sense-Approach-Take-Two/release/44815,"[23951, 1000]"
44831,Devil's Advocate,/Dave-Clarke-Devils-Advocate/master/44831,[1000]


## Artist Album ID --> Known Albums

In [28]:
from masterdb import getArtistAlbumKnownDB

In [29]:
discArtistAlbumKnownDB = getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB)
discArtistAlbumKnownDB.head()

Current Time is Wed Jan 22, 2020 21:11:41 for 
=================================== Creating Artist Album DB ===================================
Creating Pandas DataFrame for 712336 Arist Albums
	Shape --> (712336, 3)
DataFrame Shape is (712336, 3)
Current Time is Wed Jan 22, 2020 21:12:04 for Done with 
=================================== Creating Artist Album DB ===================================
Process [{0}] took 23 seconds.


,Known Albums,All Albums,Albums
1000,53,53,"{'44815': ['Archive One', 'Albums', True], '44..."
1000500,1,1,"{'1178875': ['Peak Controller', 'Miscellaneous..."
100200,9,9,"{'50511': ['PottpÃ¼ree', 'Albums', True], '219..."
1002000,3,3,"{'1002108': ['Are You Faithful?', 'Instruments..."
1002600,1,1,"{'1180905': ['Quiero', 'Vocals', True]}"


# Joining Artist ID DataFrame

In [30]:
from masterdb import createMasterDB

In [31]:
createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB)

Current Time is Wed Jan 22, 2020 21:12:04 for 
=================================== Creating Artist ID DB ===================================
Creating Pandas DataFrame for 712336 Arist IDs
  Joining Artist Metadata
	Shape --> (712336, 9)
  Joining Artist Albums
	Shape --> (712336, 12)
Current Time is Wed Jan 22, 2020 21:12:10 for Done with 
=================================== Creating Artist ID DB ===================================
Process [{0}] took 5 seconds.
Saving data to /Volumes/Music/Discog/db/MasterDB.p
  --> This file is 623.0MB.
Saved data to /Volumes/Music/Discog/db/MasterDB.p
  --> This file is 623.0MB.


In [ ]:
from random import random